In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import qmc
#import gitfame

In [ ]:
# print the gitfame data for the repo
#gitfame.main(['--sort=commits', '-wt'])

Part 1: Implementing and plotting the Mandelbrot set

In [ ]:
# hyperparameters
# bounds for the search space [x,y]
lowerBounds = [-2.0, -1.5]
upperBounds = [1.0, 1.5]

In [ ]:
def mandelbrot(c, max_iter):
    '''
    Determines whether complex number c lies within the mandelbrot set via the escape time algorithm.
    
    Args:
        c (complex):    Complex number to be tested.
        max_iter (int): Number of iterations to perform.
        
    Returns:
        z (complex): The complex number at iteration max_iter.
        n (int): The number of iterations before the escape condition is reached.
    '''
    z = 0 + 0j
    n = 0
    while abs(z) <= 2 and n < max_iter:
        z = z*z + c
        n += 1
        
    return z, n

def plot_mandelbrot(width, height, max_iter):
    '''
    Plots the Mandelbrot set at a given resolution and number of iterations.
    
    Args:
        width (int):    width of generated image in pixels.
        height (int):   height of generated image in pixels.
        max_iter (int): maximum number of iterations to be performed when calculating the set.
    
    Returns:
        None
    '''
    
    # Set arrays to store the mandelbrot set
    x, y = np.linspace(lowerBounds[0], upperBounds[0], width), np.linspace(lowerBounds[1], upperBounds[1], height)
    mandelbrot_set = np.empty((height, width))
    mandelbrot_set_bool = np.empty((height, width))
    
    # Calculate the mandelbrot set
    for i in range(height):
        for j in range(width):
            z, mandelbrot_set[i, j] = mandelbrot(x[j] + 1j * y[i], max_iter)
            if abs(z) <= 2:
                mandelbrot_set_bool[i, j] = 1
            else:
                mandelbrot_set_bool[i, j] = 0
    
    plt.figure(figsize=(10, 10))
    plt.imshow(mandelbrot_set, extent=(lowerBounds[0], upperBounds[0], lowerBounds[1], upperBounds[1]),
               cmap='inferno', interpolation='bilinear')
    plt.colorbar()
    plt.title("Mandelbrot Set")
    plt.xlabel("Re")
    plt.ylabel("Im")
    plt.show()
    
    print(mandelbrot_set_bool)
    plt.figure(figsize=(10, 10))
    plt.imshow(mandelbrot_set_bool)
    plt.colorbar()
    plt.title("Mandelbrot Set")
    plt.xlabel("Re")
    plt.ylabel("Im")
    plt.show()

In [ ]:
# Plot the Mandelbrot set with given parameters
plot_mandelbrot(500, 500, 100)

Part 2

In [ ]:
def get_random_sample(sample_size,sampling_method="random"):
    """Get nx2d random sample with various sampling methods

    Args: 
    sample_size (int)
    sampling_method (str)

    Returns:
    sampling_scaled (np.array): array of size (sampling_size, 2) of random values between lowerBounds and upperBounds
    
    """
    # use latin hypercube sampling to sample points in the square
    if sampling_method == "latin":
        sampler = qmc.LatinHypercube(d=2,strength=1)
        sample = sampler.random(sample_size)
        sample_scaled = qmc.scale(sample, lowerBounds, upperBounds)

    # number of points needs to be p**2 where p is a prime number
    if sampling_method == "orthogonal":
        sampler = qmc.LatinHypercube(d=2,strength=2)
        sample = sampler.random(sample_size)
        sample_scaled = qmc.scale(sample, lowerBounds, upperBounds)
    
    # use pure random sampling
    if sampling_method == "random":
        sample_scaled = np.random.uniform(low=lowerBounds, high=upperBounds, size=(sample_size, 2))
    
    return sample_scaled

In [ ]:
def compute_and_plot_i_errors(sample_sizes,max_iter,sampling_method):
    """Investigate errors on MC area due to finiteness of i (iterations)

    Args: 
    sample_sizes (list): list of sample sizes (int) [1000,...,50000]
    max_iter (int): max number of iterations
    sampling_method (str): choose sampling method (random, latin, orthogonal)

    Output:
    Plot of errors due to i, iterations vs (Area_i,s - Area_j,s for j in (0,..,i)) 

    """

    area_of_region = abs(upperBounds[0] - lowerBounds[0]) * abs(upperBounds[1] - lowerBounds[1]) 
    fig, ax = plt.subplots(1,1,figsize=(10,6))

    # Do for multiple runs to get a confidence interval? Or not?
    
    # check MC for one value of max_iter, do for multiple sample sizes
    for sample_size in sample_sizes:

        # get a random sample
        sample_scaled = get_random_sample(sample_size,sampling_method)
        areas = []

        # find area at each iteration 
        for iter in range(max_iter):
            mc_count = 0
                
            for i in range(sample_size):
                    z, _ = mandelbrot(sample_scaled[i][0] + 1j * sample_scaled[i][1], iter)
                    
                    if abs(z) <= 2:
                        mc_count += 1
                        
            current_area = (float(mc_count) * area_of_region) / sample_size
            areas.append(current_area)
            
        areas = np.array(areas)

        # Area_i,s - Area_j,s for j in (0,..,i)
        Ais_Ajs = np.abs(areas - areas[-1]) 
        
        ax.plot(np.arange(0,len(Ais_Ajs),1), Ais_Ajs,label=f"sample size={sample_size}")


    ax.set_yscale("log")
    ax.set_xscale("log")
    ax.legend()
    ax.set_ylabel(r"|$A_{i,s}$ - $A_{j,s}$|"+ f" j={max_iter}")
    ax.set_xlabel("iterations")
    ax.set_title(f"Errors on MC due to finiteness of i, (sampling method: {sampling_method})")
    
    plt.show()


max_iter = 300
sample_sizes = [1000,5000,10000,50000]

compute_and_plot_i_errors(sample_sizes,max_iter,sampling_method="random")

In [ ]:
def monte_carlo_mandelbrot_comparable(area_of_region, initial_samples, initial_iters, threshold, sampling_method, max_steps=20):
    """
    Monte Carlo estimation of Mandelbrot set area ensuring comparable errors for i and s.
    
    Args:
    area_of_region: Area of the sampled region in the complex plane.
    initial_samples: Starting sample size (s).
    initial_iters: Starting number of iterations (i).
    threshold: Convergence threshold for area change.
    max_steps: Maximum refinement steps.

    Returns:
    prev_area = area
    """
    s = initial_samples
    i = initial_iters
    prev_area = 0
    steps = 0
    
    steps_list = []
    area_estimates = []
    
    while steps < max_steps:
        sample_scaled = get_random_sample(s,sampling_method)
        
        # Monte Carlo
        mc_count = 0
        for point in sample_scaled:
            z, _ = mandelbrot(point[0] + 1j * point[1], i)
            if abs(z) <= 2:
                mc_count += 1
        
        current_area = (float(mc_count) * area_of_region) / s

        steps_list.append(steps)
        area_estimates.append(current_area)
        
        # Relative change due to i
        new_i = i * 10
        mc_count_i = 0
        for point in sample_scaled:
            z, _ = mandelbrot(point[0] + 1j * point[1], new_i)
            if abs(z) <= 2:
                mc_count_i += 1
        area_with_new_i = (float(mc_count_i) * area_of_region) / s
        delta_i = abs(area_with_new_i - current_area) / (current_area if current_area != 0 else 1)
        
        # Relative change due to s
        new_s = s * 10
        sample_scaled_new_s = get_random_sample(new_s,sampling_method)
        mc_count_s = 0
        for point in sample_scaled_new_s:
            z, _ = mandelbrot(point[0] + 1j * point[1], i)
            if abs(z) <= 2:
                mc_count_s += 1

        area_with_new_s = (float(mc_count_s) * area_of_region) / new_s
        delta_s = abs(area_with_new_s - current_area) / (current_area if current_area != 0 else 1)
        
        # Check if the changes are comparable
        if delta_i < threshold and delta_s < threshold and abs(delta_i - delta_s) / max(delta_i, delta_s) < 0.5:
            print(f"Converged: Area estimate = {current_area}, Samples = {s}, Iterations = {i}")
            break
        
        print(s,i)
        print(delta_s,delta_i)

        # Update values: alternate between increasing sample size and iterations
        if delta_i > delta_s:
            i = new_i
        else:
            s = new_s
        
        prev_area = current_area
        steps += 1
    
    # Plot the results
    plt.figure(figsize=(10, 6))
    plt.plot(steps_list, area_estimates, marker='o', label='Estimated Area')
    plt.axhline(y=current_area, color='r', linestyle='--', label='Converged Area')
    plt.xlabel('Refinement Steps')
    plt.ylabel('Mandelbrot Set Area')
    plt.title('Mandelbrot Set Area Estimation via Monte Carlo')
    plt.legend()
    plt.grid()
    plt.show()
    
    return current_area, s, i

area_of_region = abs(upperBounds[0] - lowerBounds[0]) * abs(upperBounds[1] - lowerBounds[1]) 
initial_samples = 10000
initial_iters = 100
threshold = 0.1

estimated_area,s,i = monte_carlo_mandelbrot_comparable(area_of_region, initial_samples, initial_iters, threshold,sampling_method="random")
print(f"Estimated Mandelbrot set area: {estimated_area} with s={s} and i={i}")

In [ ]:
# Choose a threshold for convergence and compare how the value for Area changes compared to previous Area
# Do we test convergence as a function of sample size? 
# s=100000 and i=1000

def find_convergence_samples(iter,sample_size,sampling_method):
    area_of_region = abs(upperBounds[0] - lowerBounds[0]) * abs(upperBounds[1] - lowerBounds[1])
    sample_scaled = get_random_sample(sample_size,sampling_method)
    areas = []
    mc_count = 0
    
    for i in range(1,sample_size):
        z, _ = mandelbrot(sample_scaled[i][0] + 1j * sample_scaled[i][1], iter)
                    
        if abs(z) <= 2:
            mc_count += 1
                        
        current_area = (float(mc_count) * area_of_region) / i
        areas.append(current_area)

    areas = np.array(areas)
    
    convergence = abs(np.diff(areas))[2::]

    return convergence

sampling_method="random"
sampling_method="orthogonal"
sampling_method="latin"
convergence = find_convergence_samples(1000,100000,sampling_method)

plt.yscale("log")
plt.xscale("log")
plt.plot(convergence)
plt.xlabel("sample size")
plt.ylabel(r"$A_i - A_{i-1}$")

# If we choose convergence at A_i - A(i-1) = 10**(-4)
# We now need to test the convergence rate for the different sample techniques.

In [ ]:
# Function to find convergence
def find_convergence_samples(iter, sample_size, sampling_method, threshold=1e-4, batch_size=100):
    area_of_region = abs(upperBounds[0] - lowerBounds[0]) * abs(upperBounds[1] - lowerBounds[1])
    sample_scaled = get_random_sample(sample_size, sampling_method)
    
    areas = []
    mc_count = 0
    convergence_point = None  
    
    for i in range(0, sample_size, batch_size):
        # Process samples in batches 
        batch_samples = sample_scaled[i:i + batch_size]
        for point in batch_samples:
            z, _ = mandelbrot(point[0] + 1j * point[1], iter)
            if abs(z) <= 2:
                mc_count += 1
        
        current_area = (float(mc_count) * area_of_region) / (i + batch_size)
        areas.append(current_area)
        
        # Check for convergence after at least two values
        if len(areas) > 1:
            delta = abs(areas[-1] - areas[-2])
            if delta < threshold and convergence_point is None:
                convergence_point = i + batch_size  # Sample size at convergence
    
    # Compute convergence
    areas = np.array(areas)
    convergence = abs(np.diff(areas))
    
    return convergence, convergence_point



iter = 1000
# sample_size = 100000
# sample size as (closest prime number to sqrt(100000)) ** 2, for orthogonal sampling
sample_size = 317 ** 2
sampling_methods = ["random", "orthogonal", "latin"]
threshold = 1e-4

plt.figure(figsize=(10, 6))
convergence_points = {}
for sampling_method in sampling_methods:
    convergence, convergence_point = find_convergence_samples(iter, sample_size, sampling_method, threshold)
    plt.plot(convergence, label=f"{sampling_method} (conv. @ {convergence_point})")
    convergence_points[sampling_method] = convergence_point

plt.yscale("log")
plt.xscale("log")
plt.xlabel("Sample Size")
plt.ylabel(r"$A_i - A_{i-1}$")
plt.title(f"Convergence Comparison of Sampling Methods (Threshold = {threshold})")
plt.legend()
plt.grid()
plt.show()

for method, point in convergence_points.items():
    print(f"Sampling Method: {method}, Convergence at Sample Size: {point}")